In [69]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [84]:
df = pd.read_csv('data/sign_mnist_train.csv')
imgs =df.iloc[:,1:]
y = df.iloc[:,0:1]
mean = np.mean(imgs)
stdev = np.std(imgs)
x_train = (imgs - mean) / stdev
x_train = x_train.to_numpy()
print(x_train)
print(x_train.shape)

#adding bias to x train
x_train = np.insert( x_train, 0, 1, axis = 1)
print(x_train)
print(x_train.shape)



[[-0.92895113 -0.76362507 -0.62085156 ...  0.66406257  0.65811336
   0.65493948]
 [ 0.23165213  0.2128048   0.12168008 ... -0.93157535 -0.41091344
  -0.16809718]
 [ 1.00538763  0.98894136  0.94102534 ...  0.52187701  0.51662452
   0.54623652]
 ...
 [ 0.69105758  0.63842807  0.58256179 ...  0.63246578  0.61095041
   0.62388149]
 [ 0.76359529  0.81368471  0.83860719 ... -1.54771277 -1.16552059
  -1.03772083]
 [ 0.81195376  0.78864805  0.73618903 ...  0.67986097  0.75243925
   0.8568164 ]]
(27455, 784)
[[ 1.         -0.92895113 -0.76362507 ...  0.66406257  0.65811336
   0.65493948]
 [ 1.          0.23165213  0.2128048  ... -0.93157535 -0.41091344
  -0.16809718]
 [ 1.          1.00538763  0.98894136 ...  0.52187701  0.51662452
   0.54623652]
 ...
 [ 1.          0.69105758  0.63842807 ...  0.63246578  0.61095041
   0.62388149]
 [ 1.          0.76359529  0.81368471 ... -1.54771277 -1.16552059
  -1.03772083]
 [ 1.          0.81195376  0.78864805 ...  0.67986097  0.75243925
   0.8568164 ]]
(27

In [98]:
# esta funcion construye una matrix de pesos aleatoriamente agregando una columna adicional
# de ceros para el bias
# neuron_count: numero de perceptrones
# input_count: numero de features de un elemento
def layer_w(neuron_count, input_count):
    weights = np.zeros((neuron_count, input_count+1))
    for i in range(neuron_count):
        for j in range(1, (input_count+1)):
            weights[i][j] = np.random.uniform(-0.1, 0.1)
    return weights

hidden_layer_w = layer_w(60, 784) #(60, 785)
hidden_layer_y = np.zeros(60)
hidden_layer_error = np.zeros(60)


output_layer_w = layer_w(25,60) #(25, 61)
output_layer_y = np.zeros(25)
output_layer_error = np.zeros(25)
print(output_layer_w.shape)

(25, 61)


In [22]:
chart_x = []
chart_y_train = []
chart_y_test = []
def show_learning(epoch_no, train_acc, test_acc):
    global chart_x
    global chart_y_train
    global chart_y_test
    print('epoch no:', epoch_no, ', train_acc: ', '%6.4f' % train_acc,
             ', test_acc: ', '%6.4f' % test_acc, )
    chart_x.append(epoch_no + 1)
    chart_y_test.append(1.0 - test_acc)
    chart_y_train.append(1.0 - train_acc)

def plot_learning():
    plt.plot(chart_x, chart_y_train, 'r-', label='training error')
    plt.plot(chart_x, chart_y_test, 'b-', label='test error')
    plt.axis([0, len(chart_x), 0.0, 1.0])
    plt.xlabel('training epochs')
    plt.ylabel('error')
    plt.legend()
    plt.show()

In [99]:
#parametro x es una muestra del dataset (1, n) donde n es el num de features
def forward_pass(x):
    global hidden_layer_y
    global output_layer_y

    #agregamos funcion de activacion tanh para hidden layer
    #hidden_layer.shape (n, m) donde m es el num de neuronas 
    for i,w in enumerate(hidden_layer_w):
        #z es un resultado de (1,n)*(n, 1) = (1, 1) 
        z = np.dot(x, np.transpose(w))
        hidden_layer_y[i] = np.tanh(z)
    
    #agregamos un bias
    hidden_output_arr = np.concatenate((np.array([1.0]), hidden_layer_y))
    
    #agregamos funcion de activacion sigmoide para output layer
    for i,w in enumerate(output_layer_w):
        z = np.dot(hidden_output_arr, np.transpose(w))
        output_layer_y[i] = 1.0/(1.0 + np.exp(z))

    print(output_layer_y)

#np.dot(x_train, np.transpose(hidden_layer_w)).shape



In [100]:
x = hidden_layer_w[0:1]
forward_pass(x)


[0.51365381 0.48202649 0.48360106 0.52590882 0.52358763 0.49829121
 0.51434543 0.51598422 0.47904737 0.48437918 0.50908291 0.49586454
 0.47423198 0.49983623 0.50856692 0.49374973 0.49269997 0.47535848
 0.50717977 0.47428168 0.48529041 0.502668   0.49436748 0.51049345
 0.50991961]
